In [1]:
# Imports
import os
import csv
import numpy as np
import speech_recognition as sr 
from scipy.sparse import csr_matrix
from sknetwork.path import shortest_path

from collections import defaultdict
import functools
import itertools

currentPath = os.path.dirname(os.path.abspath(''))

In [2]:
# Reading trips from csv file
# Inspired from https://stackoverflow.com/a/12398967

pathCount = 0
timeTableFileName = os.path.join(currentPath, './data/timetables_edited.csv')

# Create dictionary to associates a station name with an id
trainStationNameToId = defaultdict(functools.partial(next, itertools.count()))

with open(timeTableFileName, newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')

    # Set of the reading position (ignoring header line)
    csvfile.seek(0)
    next(reader)

    # First reading to get the number different stations and init shape of trips object
    for row in reader:
        idxA = trainStationNameToId[row[1]]
        idxB = trainStationNameToId[row[2]]
    numberOfTrainstations = len(trainStationNameToId)
    trips = np.zeros((numberOfTrainstations, numberOfTrainstations))

    # Reset of the reading position (ignoring header line)
    csvfile.seek(0)
    next(reader)

    # Reading data
    for row in reader:
        pathCount += 1 
        idxA = trainStationNameToId[row[1]]
        idxB = trainStationNameToId[row[2]]

        # If trip already exist/has already be read, display message
        indexTupple = (idxA, idxB) if idxA < idxB else (idxB, idxA)
        if trips[indexTupple] != 0:
            print(f"Trip {row[1]} - {row[2]} with a distance of {row[3]} has already be read with a distance of {trips[indexTupple]}. Ignoring the new one.")
        else:
            trips[indexTupple] = int(row[3])


# Create dictionarry to map an id to its train station name
trainStationIdToName = dict((id, name) for name, id in trainStationNameToId.items())

# Creating Compressed Sparse Row (CSR) matrix to store and work more efficiently with only the trips
tripGraph = csr_matrix(trips)

print(f"Read {tripGraph.getnnz()} trips out of {pathCount} rows for {len(trainStationNameToId)} distinct train stations.")

Read 1575 trips out of 1575 rows for 817 distinct train stations.


In [3]:
# Functions used to determine the shortest path between cities 

def getPathBetweenIds(trainStationStartId: int, trainStationEndId: int):
    global tripGraph
    return shortest_path(tripGraph, sources=trainStationStartId, targets=trainStationEndId, method='D')

def getPathBetweenCities(start: str, end: str):
    global trainStationNameToId
    # TODO (get train station IDs based on the input string and then call and return getPathBetweenIds())
    pass

def getPathForFullTrip(tripCityWaypoints: list):
    # TODO (for each trip within full trip, get the shortest path and return an array of array)
    pass   


In [4]:
# Getting user's request
# Inspired from https://www.geeksforgeeks.org/python-convert-speech-to-text-and-text-to-speech/
try: 
    r = sr.Recognizer()
    with sr.Microphone() as source: 
        print("Adjusting to noise level...")
        r.adjust_for_ambient_noise(source, duration=0.2) 
 
        print("Listening...")
        audio = r.listen(source) 
            
        print("Request recognition...")
        request = r.recognize_google(audio, language="fr-FR") 

        print(f"Parsed: '{request}'") 
            
except sr.RequestError as e: 
    print(f"Exception during request parsing: {e}") 

Adjusting to noise level...
Listening...
Request recognition...
Parsed: 'j'aimerais aller de Colmar à Lille puis à Paris'


In [5]:
# Extract cities list in the right travel order from the user's request
# TODO 

In [6]:
# Call getPathForFullTrip() and display the results
# TODO